# ニューラルネットワークの学習

In [10]:
%matplotlib inline
import sys, os
sys.path.append("../deep-learning-from-scratch/")
import matplotlib.pyplot as plt
import numpy as np
from common.functions import *
from common.gradient import numerical_gradient
from dataset.mnist import load_mnist

# 1層のニューラルネットの実装

In [11]:
class simpleNet:
    def __init__(self):
        self.W = np.random.randn(2,3)
        
    def predict(self, x):
        return np.dot(x, self.W)
    
    def loss(self, x, t):
        z = self.predict(x)
        y = softmax(z)
        loss = cross_entropy_error(y, t)
        
        return loss

In [12]:
#test simpleNet
net = simpleNet()
print("W = {}\n".format(net.W))

x = np.array([0.6, 0.9])
p = net.predict(x)
print("p = {}\n".format(p))
print("label = {}\n".format(np.argmax(p)))

t = np.array([0, 0, 1])
print("output loss function = {}\n".format(net.loss(x, t)))

dW = numerical_gradient(lambda w: net.loss(x, t) , net.W)
print("dW = {}\n".format(dW))

W = [[ 0.24819631  0.90953413  0.34194568]
 [-0.75787727 -1.90600955  0.98926578]]

p = [-0.53317176 -1.16968811  1.09550661]

label = 2

output loss function = 0.26236311880727664

dW = [[ 0.09054873  0.04791228 -0.13846101]
 [ 0.1358231   0.07186842 -0.20769151]]



# 2層のユーラルネットワークの実装

In [13]:
class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        self.params = {}
        self.params['W1'] = weight_init_std*np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std*np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
        
    def predict(self, x):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
        
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)
        return y
    
    def loss(self, x, t):
        y = self.predict(x)
        return cross_entropy_error(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)
        accuracy = np.sum(y == t) / float(x.shape[0])
        
        return accuracy
    
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        return grads
    
    def gradient(self, x, t):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
        grads = {}
        batch_num = x.shape[0]
        
        #forward
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)
        
        #backward
        dy = (y - t) / batch_num
        grads['W2'] = np.dot(z1.T, dy)
        grads['b2'] = np.sum(dy, axis=0)
        
        da1 = np.dot(dy, W2.T)
        dz1 = sigmoid_grad(a1) * da1
        grads['W1'] = np.dot(x.T, dz1)
        grads['b1'] = np.sum(dz1, axis=0)
        return grads
    

In [14]:
#test TwoLayerNet
net = TwoLayerNet(input_size=784, hidden_size=100, output_size=10)
print("shape of W1 = {}".format(net.params['W1'].shape))
print("shape of b1 = {}".format(net.params['b1'].shape))
print("shape of W2 = {}".format(net.params['W2'].shape))
print("shape of b2 = {}".format(net.params['b2'].shape))

x = np.random.rand(100, 784) #ダミーの入力ラベル 
t = np.random.rand(100, 10) #ダミーの正解ラベル
grads = net.gradient(x, t)
print("shape of W1 gradient = {}".format(grads['W1'].shape))
print("shape of b1 gradient = {}".format(grads['b1'].shape))
print("shape of W2 gradient = {}".format(grads['W2'].shape))
print("shape of b2 gradient = {}".format(grads['b2'].shape))

shape of W1 = (784, 100)
shape of b1 = (100,)
shape of W2 = (100, 10)
shape of b2 = (10,)
shape of W1 gradient = (784, 100)
shape of b1 gradient = (100,)
shape of W2 gradient = (100, 10)
shape of b2 gradient = (10,)


# 数値微分を用いた、2層のニューラルネットワークでの学習

In [15]:
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)
train_loss_list = []
train_acc_list = []
test_acc_list = []

#ハイパーパラメータ 
iters_num = 10000
train_size = x.shape[0]
batch_size = 100
learning_rate = 0.1

#1エポックあたりの繰り返し数 
iter_per_epoch = max(train_size / batch_size, 1)

network = TwoLayerNet(input_size=784, hidden_size=100, output_size=10)

for i in range(iters_num):
    #ミニバッチの取得 
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    #勾配の計算 
    grads = network.gradient(x_batch, t_batch)

    #パラメータの更新 
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grads[key]
        
    #学習経過の記録 
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    #1エポックごとに認識精度を計算
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
    
print("tarin loss = {}".format(train_loss_list))
print("train accuracy = {}".format(train_acc_list))
print("test accuracy = {}".format(test_acc_list))

tarin loss = [2.2573820599340855, 2.248374895587554, 2.2522579016447768, 2.2412484478873433, 2.2483849742559143, 2.2022757143124307, 2.2455283610736192, 2.2832688446607095, 2.2756024628165683, 2.2306576801403222]
train accuracy = [0.098716666666666661, 0.11236666666666667, 0.11236666666666667, 0.11236666666666667, 0.11236666666666667, 0.098716666666666661, 0.11236666666666667, 0.11236666666666667, 0.11236666666666667, 0.11236666666666667]
test accuracy = [0.098000000000000004, 0.1135, 0.1135, 0.1135, 0.1135, 0.098000000000000004, 0.1135, 0.1135, 0.1135, 0.1135]
